In [1]:
import pandas as pd
import numpy as np
import joblib

scaler = joblib.load('minmaxscaler.pkl')
pca = joblib.load('pcamodel.pkl')



C:\Users\dell\anaconda3\envs\deeplearning\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\dell\anaconda3\envs\deeplearning\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.6.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
single_row = [
    88,640,7,4,440,358,220,0,62.85714286,107.349008,179,0,89.5,103.3456982,
    1246875,17187.5,64,135.557286,445,1,640,106.6666667,194.3251571,497,1,538,
    179.3333333,303.6862416,530,4,0,0,0,0,164,104,10937.5,6250,0,220,66.5,
    99.00183653,9801.363636,0,0,0,1,0,0,0,0,0,72.54545455,62.85714286,89.5,
    164,0,0,0,0,0,0,7,440,4,358,8192,2053,2,20,0,0,0,0,0,0,0,0
]

# Create DataFrame - IMPORTANT: same number and order of columns as training features
X_single = pd.DataFrame([single_row], columns=[col for col in scaler.feature_names_in_])

# Handle infinities and NaNs (same as training pipeline)
X_single.replace([np.inf, -np.inf], np.nan, inplace=True)
X_single.fillna(X_single.mean(), inplace=True)

# Apply scaling
X_scaled = scaler.transform(X_single)

# Apply PCA
X_pca = pca.transform(X_scaled)

# Convert to DataFrame
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
single_pca_df = pd.DataFrame(X_pca, columns=pca_columns)

C:\Users\dell\anaconda3\envs\deeplearning\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


In [3]:
single_pca_df

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC41,PC42,PC43,PC44,PC45,PC46,PC47,PC48,PC49,PC50
0,-0.048867,-0.732472,0.273533,0.002962,-0.072613,0.013336,-0.044247,0.113164,-0.123968,-0.221363,...,0.002193,0.000866,0.001028,-0.002825,0.00197,0.001408,0.000006,0.00013,-0.000039,0.000021


In [4]:
X = single_pca_df.values   
Xtest = np.array(X)
Xtest = np.reshape(Xtest, (Xtest.shape[0], Xtest.shape[1], 1))

In [5]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
# Define early stopping and learning rate scheduler callbacks
initial_learning_rate = 0.001
def lr_schedule(epoch):
    return initial_learning_rate * (0.1 ** (epoch // 10))

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(lr_schedule)

# Define the rest of your callbacks
my_callbacks = [
    ModelCheckpoint(filepath='cnnlstm.h5', verbose=1, save_best_only=True, monitor='val_accuracy', mode='max'),
    early_stopping,
    lr_scheduler
]
model = Sequential()

# Add Convolutional and MaxPooling Layers with L2 Regularization
model.add(Conv1D(64, 3, padding="same", input_shape=(Xtest.shape[1], 1), activation='relu', kernel_regularizer=l2(0.01)))
model.add(Conv1D(64, 3, padding="same", activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, 3, padding="same", activation='relu', kernel_regularizer=l2(0.001)))
model.add(Conv1D(128, 3, padding="same", activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling1D(pool_size=2))

# Add Batch Normalization
model.add(BatchNormalization())

# Add LSTM layer with L2 Regularization
model.add(LSTM(units=100, return_sequences=False, dropout=0.1, kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))  # Add dropout to reduce overfitting

# Add Dense layer with L2 Regularization
model.add(Dense(units=2, activation='softmax', kernel_regularizer=l2(0.01))) 

# Compile the model with a custom learning rate schedule
opt = Adam(learning_rate=initial_learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 50, 64)            256       
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 64)            12352     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 25, 64)           0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 25, 128)           24704     
                                                                 
 conv1d_3 (Conv1D)           (None, 25, 128)           49280     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 12, 128)          0         
 1D)                                                    

In [6]:
model.load_weights('model.h5')

In [8]:
Y_pred = model.predict(Xtest)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 


In [9]:
import pickle
with open('LIME.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

C:\Users\dell\anaconda3\envs\deeplearning\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\dell\anaconda3\envs\deeplearning\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
y_predrf = loaded_model.predict_proba(X)

C:\Users\dell\anaconda3\envs\deeplearning\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [11]:
fused_probs = (Y_pred + y_predrf) / 2
Y_pred_classes = np.argmax(fused_probs,axis = 1) 

In [13]:
if(Y_pred_classes[0]==0):
    print("Normal")
else:
    print("Malicious")

Normal
